In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
class NeuralNetwork(nn.Module):
    def __init__ (self,input_size,hidden_size,num_classes):
        super(NeuralNetwork,self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size,hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.LeakyReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(hidden_size,hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.LeakyReLU(),
            
            nn.Linear(hidden_size,hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(hidden_size,hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.LeakyReLU(),
            
            nn.Linear(hidden_size,hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.LeakyReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(hidden_size,hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.LeakyReLU(),
            
            nn.Linear(hidden_size,num_classes)
        )
        
    def forward(self,x):
        return self.layers(x)
            

In [4]:
df = pd.read_csv('/Users/hari/Downloads/Maternal Health Risk Data Set.csv.xls')
df

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.0,98.0,86,high risk
1,35,140,90,13.0,98.0,70,high risk
2,29,90,70,8.0,100.0,80,high risk
3,30,140,85,7.0,98.0,70,high risk
4,35,120,60,6.1,98.0,76,low risk
...,...,...,...,...,...,...,...
1009,22,120,60,15.0,98.0,80,high risk
1010,55,120,90,18.0,98.0,60,high risk
1011,35,85,60,19.0,98.0,86,high risk
1012,43,120,90,18.0,98.0,70,high risk


In [5]:
risk_level_map = {'high risk':2,'mid risk':1,'low risk':0}
df['RiskLevel'] = df['RiskLevel'].map(risk_level_map)

In [6]:
df

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.0,98.0,86,2
1,35,140,90,13.0,98.0,70,2
2,29,90,70,8.0,100.0,80,2
3,30,140,85,7.0,98.0,70,2
4,35,120,60,6.1,98.0,76,0
...,...,...,...,...,...,...,...
1009,22,120,60,15.0,98.0,80,2
1010,55,120,90,18.0,98.0,60,2
1011,35,85,60,19.0,98.0,86,2
1012,43,120,90,18.0,98.0,70,2


In [7]:
train_data , test_data = train_test_split(df,random_state=42,shuffle=True,test_size=0.2)

In [8]:
train_data

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
847,30,120,80,9.0,101.0,76,1
332,23,130,70,6.9,98.0,70,1
707,32,120,90,6.9,98.0,70,1
218,31,120,60,6.1,98.0,76,1
425,35,100,60,15.0,98.0,80,2
...,...,...,...,...,...,...,...
106,50,140,90,15.0,98.0,90,2
270,22,120,90,7.0,98.0,82,0
860,17,90,63,6.9,101.0,70,1
435,65,130,80,15.0,98.0,86,2


In [9]:
test_data

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
752,29,130,70,7.70,98.0,78,1
519,30,140,100,15.00,98.0,70,2
210,50,140,95,17.00,98.0,60,2
611,23,120,90,7.50,98.0,60,0
914,17,120,80,7.50,102.0,76,0
...,...,...,...,...,...,...,...
370,21,75,50,7.80,98.0,60,0
381,23,120,90,7.80,98.0,60,1
780,60,90,65,6.80,98.0,77,1
760,21,90,60,6.90,98.0,86,1


In [10]:
X_train = train_data.drop(columns=['RiskLevel','BodyTemp'])
y_train = train_data['RiskLevel']

In [11]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_resampled,y_train_resampled = smote.fit_resample(X_train,y_train)

In [12]:
import numpy as np
from collections import Counter

print("Before SMOTE:", Counter(y_train))
print("After SMOTE: ", Counter(y_train_resampled))

Before SMOTE: Counter({0: 326, 1: 260, 2: 225})
After SMOTE:  Counter({1: 326, 2: 326, 0: 326})


In [13]:
X_test = test_data.drop(columns=['RiskLevel','BodyTemp'])
y_test = test_data['RiskLevel']

In [14]:
scaler = StandardScaler()
X_train_resampled = scaler.fit_transform(X_train_resampled)
X_test = scaler.fit_transform(X_test)

In [15]:
X_train_resampled = torch.tensor(X_train_resampled,dtype=torch.float32)
X_test = torch.tensor(X_test,dtype=torch.float32)

In [16]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_resampled = le.fit_transform(y_train_resampled)  
y_test = le.transform(y_test)

In [17]:
import torch

y_train_resampled = torch.tensor(y_train_resampled, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [18]:
y_train_resampled = torch.tensor(np.array(y_train_resampled),dtype=torch.long)
y_test = torch.tensor(np.array(y_test),dtype=torch.long)

/var/folders/vk/1wl47y5s0y11p511c75cq_2r0000gn/T/ipykernel_21944/3150256865.py:1: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  y_train_resampled = torch.tensor(np.array(y_train_resampled),dtype=torch.long)
/var/folders/vk/1wl47y5s0y11p511c75cq_2r0000gn/T/ipykernel_21944/3150256865.py:2: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  y_test = torch.tensor(np.array(y_test),dtype=torch.long)


In [26]:
from skorch import NeuralNetClassifier

# Wrap your PyTorch model
skorch_model = NeuralNetClassifier(
    module=NeuralNetwork,
    module__input_size=input_size,
    module__hidden_size=hidden_size,
    module__num_classes=num_classes,
    max_epochs=5000,
    lr=0.001,
    optimizer=torch.optim.Adam,
    criterion=nn.CrossEntropyLoss,
    iterator_train__shuffle=True,
    verbose=0,
    device='cuda' if torch.cuda.is_available() else 'cpu'
)


In [27]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)


In [28]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

stack = StackingClassifier(
    estimators=[
        ('xgb', xgb_model),
        ('rf', rf_model),
        ('nn', skorch_model),
    ],
    final_estimator=LogisticRegression(),
    cv=5,
    passthrough=True  # optional: includes original features too
)


In [ ]:
stack.fit(X_train_resampled, y_train_resampled)


/opt/anaconda3/envs/MachineLearning/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:26:29] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [25]:
from sklearn.metrics import classification_report, roc_auc_score

y_pred = stack.predict(X_test)
y_proba = stack.predict_proba(X_test)

print(classification_report(y_test, y_pred))
print("AUROC:", roc_auc_score(y_test, y_proba, multi_class="ovr"))


              precision    recall  f1-score   support

           0       0.66      0.51      0.58        80
           1       0.64      0.58      0.61        76
           2       0.58      0.89      0.71        47

    accuracy                           0.63       203
   macro avg       0.63      0.66      0.63       203
weighted avg       0.63      0.63      0.62       203

AUROC: 0.8423291197818127
